In [1]:
import warnings
warnings.filterwarnings('ignore')
from product_details import Parameters
pm = Parameters()


In [38]:
# As-Is Situation

# Define the as-is DC allocation. The keys are the DC, the values are the states where the corresponding DC's customers are.
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

all_costs = pm.total_costs(as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Handling In Costs:', round(all_costs[2]))
print('Total Storage Costs:', round(all_costs[3]))
print('Total Handling Out Costs:', round(all_costs[4]))
print('Total Outbound Costs:', round(all_costs[5]))
print('Total Operational Costs:', round(all_costs[6]))

Total Costs: 11363602
Total Inbound Costs: 1124750
Total Handling In Costs: 55780
Total Storage Costs: 1341612
Total Handling Out Costs: 686104
Total Outbound Costs: 1155356
Total Operational Costs: 7000000


In [4]:
# Algorithm 1: Greedy Constructive Search
# Allocate dc based on the distance/costs greedily 
greedy_dc_allocation = {}
outbound_data = pm.outbound_data
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

print(greedy_dc_allocation)

all_costs = pm.total_costs(greedy_dc_allocation, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Handling In Costs:', round(all_costs[2]))
print('Total Storage Costs:', round(all_costs[3]))
print('Total Handling Out Costs:', round(all_costs[4]))
print('Total Outbound Costs:', round(all_costs[5]))
print('Total Operational Costs:', round(all_costs[6]))

# Turns out, even though the shipping costs are lower,
# the total outbound costs are relatively higher since the operating and opening costs are skyrocketed.

{'WA': ['AK', 'ID', 'MT', 'OR', 'WA'], 'TN': ['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'OH', 'SC', 'VA', 'WV'], 'TX': ['AR', 'LA', 'NM', 'OK', 'TN', 'TX'], 'UT': ['AZ', 'CO', 'NV', 'UT', 'WY'], 'CA': ['CA'], 'NY': ['CT', 'DE', 'MA', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT'], 'PA': ['HI', 'MD', 'PA', 'DC'], 'KS': ['IA', 'KS', 'MO', 'NE'], 'IL': ['IL', 'MI', 'MN'], 'ND': ['IN', 'ND', 'SD', 'WI']}
Total Costs: 15615620
Total Inbound Costs: 1215700
Total Handling In Costs: 60340
Total Storage Costs: 1278576
Total Handling Out Costs: 697577
Total Outbound Costs: 933427
Total Operational Costs: 11430000


In [66]:
# Algorithm 1b: Greedy Constructive Heuristics
# Now, knowing that opening/closing and operating new DC can be costly, we apply the same algorithm without opening a new DC.

greedy_dc_allocation = {}
outbound_data = pm.outbound_data
as_is = list(as_is_dc.keys())
as_is.remove('WA')
as_is.remove('ND')
as_is.remove('UT')



for state in outbound_data.index:
    current_dc = outbound_data[as_is]
    best_dc = (current_dc.loc[state]).idxmin()
    
    # if best_dc not in as_is:
    #     print(best_dc)
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

all_costs = pm.total_costs(greedy_dc_allocation, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Handling In Costs:', round(all_costs[2]))
print('Total Storage Costs:', round(all_costs[3]))
print('Total Handling Out Costs:', round(all_costs[4]))
print('Total Outbound Costs:', round(all_costs[5]))
print('Total Operational Costs:', round(all_costs[6]))
print(greedy_dc_allocation)
# The shipping costs are lowered again, but the handling costs are increasing slightly.
# It resulted in a less significant decrease.

Total Costs: 11395697
Total Inbound Costs: 1131350
Total Handling In Costs: 55260
Total Storage Costs: 1099380
Total Handling Out Costs: 704197
Total Outbound Costs: 1380510
Total Operational Costs: 7025000
{'NY': ['AK', 'CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'TN': ['AL', 'AR', 'AZ', 'CA', 'FL', 'GA', 'ID', 'KS', 'KY', 'MO', 'MS', 'NC', 'NE', 'OH', 'PA', 'SC', 'TN', 'UT', 'VA', 'WV'], 'TX': ['CO', 'LA', 'NM', 'NV', 'OK', 'TX'], 'IL': ['IA', 'IL', 'IN', 'MI', 'MN', 'MT', 'ND', 'OR', 'SD', 'WA', 'WI', 'WY']}


Best DC to Remove: ND
Total Costs: 15627236
Total Inbound Costs: 2687900
Total Handling In Costs: 109500
Total Storage Costs: 2199180
Total Handling Out Costs: 1435827
Total Outbound Costs: 3169828
Total Operational Costs: 6025000
Best Greedy DC Allocation: {'NY': ['AR', 'NY', 'ME', 'VT', 'CT', 'DE', 'MA', 'NJ', 'NY'], 'IL': ['AK', 'ME', 'MT', 'VT', 'IL', 'IN', 'KY', 'MD', 'MI', 'MN', 'NH', 'OH', 'RI', 'SD', 'VA', 'WI', 'OR', 'WA', 'AK', 'MT'], 'TN': ['AL', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MA', 'MD', 'MI', 'MN', 'MO', 'MS', 'NC', 'NH', 'NJ', 'OH', 'PA', 'RI', 'SC', 'SD', 'TN', 'VA', 'WI', 'WV', 'WY', 'AR', 'NY', 'LA', 'NM', 'OK', 'TX', 'DC', 'AL', 'FL', 'GA', 'MO', 'MS', 'NC', 'SC', 'TN', 'WV', 'AR', 'CA', 'CO', 'ID', 'ND', 'NE', 'UT', 'HI', 'IA', 'KS', 'PA', 'WY'], 'WA': [], 'TX': ['AZ', 'LA', 'NM', 'OK', 'TX', 'DC', 'AL', 'FL', 'GA', 'MO', 'MS', 'NC', 'SC', 'TN', 'WV', 'AR', 'NV', 'AZ'], 'UT': ['CA', 'CO', 'ID', 'ND', 'NE', 'NV', 'OR', 'UT', 'WA', 'AK', 'M

In [6]:
# Best solution (Simulated Annealing)
to_be = {'WA': ['AK', 'WA'],
                'TN': ['PA', 'MD', 'TN', 'OH', 'AL', 'DC', 'RI', 'ME', 'WV', 'NJ', 'HI', 'NC', 'KY', 'MS', 'GA', 'KS', 'IL', 'FL', 'MI', 'AR', 'MA', 'VA', 'CT', 'SC', 'DE', 'MO', 'NY'],
                'UT': ['NV', 'ID', 'CA', 'AZ', 'UT', 'CO', 'WY'],
                'NY': ['VT'],
                'ND': ['ND', 'MN', 'OR', 'IA', 'NE', 'MT', 'WI', 'SD', 'IN'],
                'IL': ['NH'], 'TX': ['LA', 'OK', 'NM', 'TX']}

all_costs = pm.total_costs(to_be, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))
print(greedy_dc_allocation)

Total Costs: 10883825
Total Inbound Costs: 1191250
Total Warehousing Costs: 38490
Total Outbound Costs: 668772
Total Operational Costs: 737088
{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}


In [25]:
dc = {'NY': ['DE', 'NY'],
      'ND': ['ID', 'MN', 'ND', 'WA', 'MT', 'SD', 'WI', 'IN'],
      'IL': ['NH', 'ME', 'DC'],
      'TN': ['WV', 'KY', 'NJ', 'HI', 'MS', 'IL', 'AL', 'PA', 'VA', 'MA', 'CT', 'SC', 'AR', 'NE', 'NC', 'GA', 'OH', 'KS', 'MO', 'VT', 'RI', 'FL', 'MI', 'IA', 'MD', 'TN'],
      'WA': [], 'TX': ['LA', 'OK', 'NM', 'TX'], 'UT': ['WY', 'NV', 'CA', 'AZ', 'AK', 'OR', 'CO', 'UT']}

all_costs = pm.total_costs(dc, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))
print(greedy_dc_allocation)

Total Costs: 10465306
Total Inbound Costs: 1155200
Total Warehousing Costs: 38820
Total Outbound Costs: 796956
Total Operational Costs: 731281
{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}
